<a href="https://colab.research.google.com/github/skywalker0803r/RL/blob/main/%E5%9C%88%E5%9C%88%E5%8F%89%E5%8F%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# --- 1. 遊戲環境 (Game Environment) ---

class TicTacToe:
    def __init__(self):
        # 棋盤初始化為 3x3 的零矩陣
        self.board = np.zeros((3, 3), dtype=int)
        # 1 代表 'X' (Agent)，-1 代表 'O' (Opponent/Human)
        self.player_symbol = 1
        self.is_end = False

    def get_state(self):
        """將當前棋盤狀態轉換為一個唯一的、可雜湊的整數或字串作為狀態標識。"""
        # 將 3x3 矩陣展平為 9 個數字，轉換為 tuple
        return tuple(self.board.flatten())

    def check_win(self, board):
        """檢查是否有玩家獲勝。"""
        # 檢查行、列、對角線的總和是否為 3 (X 獲勝) 或 -3 (O 獲勝)
        for i in range(3):
            # 行檢查
            if np.sum(board[i, :]) == 3: return 1  # X Win
            if np.sum(board[i, :]) == -3: return -1 # O Win
            # 列檢查
            if np.sum(board[:, i]) == 3: return 1
            if np.sum(board[:, i]) == -3: return -1

        # 對角線檢查
        if board[0, 0] + board[1, 1] + board[2, 2] == 3: return 1
        if board[0, 0] + board[1, 1] + board[2, 2] == -3: return -1
        if board[0, 2] + board[1, 1] + board[2, 0] == 3: return 1
        if board[0, 2] + board[1, 1] + board[2, 0] == -3: return -1

        # 檢查是否平局 (沒有空格子且沒有獲勝者)
        if np.sum(board == 0) == 0: return 0  # Draw

        return None # 遊戲尚未結束

    def available_positions(self):
        """返回所有可下棋子的空閒位置 (row, col)。"""
        return list(zip(*np.where(self.board == 0)))

    def step(self, position):
        """執行一個行動，並返回結果 (獎勵和遊戲是否結束)。"""
        if self.board[position] != 0:
            # 非法行動，但在訓練中通常不會發生，我們會確保 Agent 只選有效動作
            return 0, True

        # 執行動作
        self.board[position] = self.player_symbol
        reward = 0
        is_end = False

        # 檢查遊戲結果
        win_status = self.check_win(self.board)
        if win_status is not None:
            is_end = True
            # 獎勵： Agent(X) 獲勝為 1, 失敗為 -1, 平局為 0
            if self.player_symbol == 1:
                reward = win_status
            else:
                reward = -win_status # 如果是 O 的回合，獎勵與獲勝狀態相反

        # 切換玩家
        self.player_symbol *= -1
        self.is_end = is_end

        return reward, is_end

# --- 2. 強化學習代理人 (RL Agent) ---

class Agent:
    def __init__(self, name="X", exp_rate=0.3, lr=0.2):
        self.name = name
        # 狀態價值表 V(S): 儲存每個狀態的預期價值
        self.V = {}
        # 探索率 (epsilon): 用於平衡 探索 (exploration) 和 利用 (exploitation)
        self.exp_rate = exp_rate
        # 學習率 (alpha): 決定每次更新的步長
        self.lr = lr
        # 儲存一局遊戲中的所有狀態，用於回溯更新
        self.states_path = []

    def choose_action(self, env):
        """根據 epsilon-greedy 策略選擇行動。"""
        available_pos = env.available_positions()

        if np.random.uniform(0, 1) <= self.exp_rate:
            # 探索 (Exploration): 隨機選擇一個合法動作
            idx = np.random.choice(len(available_pos))
            action = available_pos[idx]
        else:
            # 利用 (Exploitation): 選擇能導向價值最高狀態的動作
            best_action = None
            max_value = -999

            for pos in available_pos:
                # 模擬下一個狀態
                next_board = env.board.copy()
                next_board[pos] = env.player_symbol
                next_state = tuple(next_board.flatten())

                # 獲取該狀態的價值 (如果未見過，則預設為 0.5)
                value = self.V.get(next_state, 0.5)

                if value > max_value:
                    max_value = value
                    best_action = pos

            action = best_action

        return action

    def feed_state(self, state):
        """記錄當前狀態，用於後續的價值更新。"""
        self.states_path.append(state)
        # 初始化未見過狀態的價值
        if state not in self.V:
            self.V[state] = 0.5

    def learn(self, final_reward):
        """
        遊戲結束後，使用 TD 規則回溯更新路徑上的所有狀態價值。
        TD Update: V(S_t) <- V(S_t) + α * [V(S_{t+1}) - V(S_t)]

        我們從終點開始，將最終獎勵 (R) 傳播回去。
        對於最後一步: V(S_{T-1}) <- V(S_{T-1}) + α * [R - V(S_{T-1})]
        對於前一步: V(S_{t}) <- V(S_{t}) + α * [V(S_{t+1}) - V(S_{t})]
        """

        # 從終點狀態開始反向遍歷
        target = final_reward
        for state in reversed(self.states_path):
            current_value = self.V[state]

            # TD 更新
            new_value = current_value + self.lr * (target - current_value)
            self.V[state] = new_value

            # 將目標值更新為當前狀態的新價值，作為前一個狀態的預期回報 (Target)
            target = new_value

        # 清空路徑，準備下一局遊戲
        self.states_path = []

# --- 3. 訓練與對戰功能 (Training and Play) ---

def train(agent_x, agent_o, episodes=50000):
    """讓兩個 AI 代理人相互對戰進行訓練。"""
    print(f"--- 開始訓練：{episodes} 局 ---")
    results = {'X_win': 0, 'O_win': 0, 'Draw': 0}

    for i in range(episodes):
        game = TicTacToe()

        while not game.is_end:
            current_state = game.get_state()

            if game.player_symbol == 1: # Agent X 的回合
                agent_x.feed_state(current_state)
                action = agent_x.choose_action(game)
            else: # Agent O 的回合 (也需要學習)
                agent_o.feed_state(current_state)
                action = agent_o.choose_action(game)

            reward, is_end = game.step(action)

            if is_end:
                # 遊戲結束，根據結果更新兩個 Agent
                if reward == 1:
                    results['X_win'] += 1
                elif reward == -1:
                    results['O_win'] += 1
                else:
                    results['Draw'] += 1

                # X 的獎勵是 +1, O 的獎勵是 -1 (或平局 0)
                agent_x.learn(reward)
                agent_o.learn(-reward) # O 的獎勵是 X 的獎勵的反向

        if (i + 1) % 10000 == 0:
            print(f"已完成 {i + 1} 局。 價值表大小: {len(agent_x.V)} 個狀態")

    print(f"--- 訓練結束 ---")
    print(f"X 獲勝: {results['X_win']}, O 獲勝: {results['O_win']}, 平局: {results['Draw']}")
    print(f"X 最終探索率: {agent_x.exp_rate:.2f}")

def play_vs_human(trained_agent_x):
    """讓訓練好的 Agent X 與人類玩家 O 對戰。"""
    # 對戰時，Agent X 應該關閉探索 (exp_rate=0)，完全依照學到的知識 (V table) 進行利用
    trained_agent_x.exp_rate = 0

    while True:
        game = TicTacToe()
        print("\n--- 新遊戲開始 ---")

        while not game.is_end:
            print("-" * 20)
            print(f"當前棋盤 (1=X, -1=O): \n{game.board}")

            if game.player_symbol == 1: # Agent X 的回合
                print("AI (X) 正在思考...")
                # 獲取當前狀態，讓 Agent 選擇最佳行動
                action = trained_agent_x.choose_action(game)

                reward, is_end = game.step(action)
                print(f"AI (X) 下在 ({action[0]}, {action[1]})")

            else: # 人類玩家 O 的回合
                print("輪到你 (O) 了。可選位置:", game.available_positions())
                try:
                    r = int(input("請輸入行 (0-2): "))
                    c = int(input("請輸入列 (0-2): "))
                    if (r, c) not in game.available_positions():
                        print("位置無效或已被佔用，請重新輸入。")
                        continue
                    action = (r, c)
                    reward, is_end = game.step(action)
                except ValueError:
                    print("輸入無效，請輸入數字。")
                    continue

            if is_end:
                final_status = game.check_win(game.board)
                print("-" * 20)
                print(f"最終棋盤: \n{game.board}")
                if final_status == 1:
                    print("恭喜 AI (X) 獲勝！🎉")
                elif final_status == -1:
                    print("恭喜你 (O) 獲勝！👏")
                else:
                    print("平局。🤝")
                break

        if input("再玩一局嗎? (y/n): ").lower() != 'y':
            break

# --- 4. 執行流程 ---

if __name__ == "__main__":
    # 初始化兩個 Agent，進行相互學習
    # 讓 X 的學習率高一點，O 的探索率低一點，模擬 X 是「主角」
    agent_x = Agent(name="X", exp_rate=0.3, lr=0.2)
    agent_o = Agent(name="O", exp_rate=0.1, lr=0.1)

    # 1. 訓練 (訓練 50000 局，Agent X 和 Agent O 都會學習)
    train(agent_x, agent_o, episodes=50000)

    # 2. 對戰 (使用訓練好的 Agent X 與人類玩家 O 對戰)
    play_vs_human(agent_x)

--- 開始訓練：50000 局 ---
已完成 10000 局。 價值表大小: 625 個狀態
已完成 20000 局。 價值表大小: 763 個狀態
已完成 30000 局。 價值表大小: 853 個狀態
已完成 40000 局。 價值表大小: 901 個狀態
已完成 50000 局。 價值表大小: 935 個狀態
--- 訓練結束 ---
X 獲勝: 45378, O 獲勝: 0, 平局: 4622
X 最終探索率: 0.30

--- 新遊戲開始 ---
--------------------
當前棋盤 (1=X, -1=O): 
[[0 0 0]
 [0 0 0]
 [0 0 0]]
AI (X) 正在思考...
AI (X) 下在 (0, 0)
--------------------
當前棋盤 (1=X, -1=O): 
[[1 0 0]
 [0 0 0]
 [0 0 0]]
輪到你 (O) 了。可選位置: [(np.int64(0), np.int64(1)), (np.int64(0), np.int64(2)), (np.int64(1), np.int64(0)), (np.int64(1), np.int64(1)), (np.int64(1), np.int64(2)), (np.int64(2), np.int64(0)), (np.int64(2), np.int64(1)), (np.int64(2), np.int64(2))]
請輸入行 (0-2): 1
請輸入列 (0-2): 1
--------------------
當前棋盤 (1=X, -1=O): 
[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  0]]
AI (X) 正在思考...
AI (X) 下在 (0, 1)
--------------------
當前棋盤 (1=X, -1=O): 
[[ 1  1  0]
 [ 0 -1  0]
 [ 0  0  0]]
輪到你 (O) 了。可選位置: [(np.int64(0), np.int64(2)), (np.int64(1), np.int64(0)), (np.int64(1), np.int64(2)), (np.int64(2), np.int64(0)), (np.int64(2), 